In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
import numpy as np

# 하이퍼파라미터 설정
batch_size = 16
learning_rate = 0.001
num_epochs = 10

# load_digits 데이터셋 로드
digits = load_digits()
X_full, y_full = digits.data, digits.target

# 숫자 1과 7에 해당하는 데이터만 필터링
class1_indices = np.where(y_full == 1)[0][:100]
class7_indices = np.where(y_full == 7)[0][:100]
selected_indices = np.concatenate([class1_indices, class7_indices])

X_filtered = X_full[selected_indices]
y_filtered = y_full[selected_indices]

# 레이블 이진화: 1 -> 0, 7 -> 1
y_filtered = (y_filtered == 7).astype(int)

# 데이터 정규화 (0-16 값을 0-1 사이로)
X_filtered = X_filtered / 16.0

# 데이터 차원 변환: CNN 입력에 맞게 8x8 이미지 형태로 재구성
X_filtered = X_filtered.reshape(-1, 8, 8)

# Stratify 방식으로 train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_filtered, y_filtered, stratify=y_filtered, test_size=0.2, random_state=42
)

# PyTorch Tensor로 변환
X_train = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)  # 채널 차원 추가
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
y_test = torch.tensor(y_test, dtype=torch.float32)

# 데이터셋과 데이터 로더 정의
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

# CNN 모델 정의
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 2 * 2, 128)
        self.fc2 = nn.Linear(128, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(x)
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = x.view(-1, 64 * 2 * 2)
        x = self.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# 모델 초기화
model = CNNModel()
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 학습 루프
for epoch in range(num_epochs):
    model.train()
    train_correct = 0
    train_total = 0
    for images, labels in train_loader:
        labels = labels.unsqueeze(1)  # BCELoss에 맞추어 차원 맞추기

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accuracy 계산
        predicted = (outputs > 0.5).float()
        train_correct += (predicted == labels).sum().item()
        train_total += labels.size(0)

    train_accuracy = 100 * train_correct / train_total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}, Accuracy: {train_accuracy:.2f}%')


/home/jaeyeob/anaconda3/envs/lstm/lib/python3.10/site-packages/torch/autograd/graph.py:825: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch [1/10], Loss: 0.6397, Accuracy: 60.00%
Epoch [2/10], Loss: 0.4801, Accuracy: 89.38%
Epoch [3/10], Loss: 0.2488, Accuracy: 98.12%
Epoch [4/10], Loss: 0.1169, Accuracy: 97.50%
Epoch [5/10], Loss: 0.0729, Accuracy: 100.00%
Epoch [6/10], Loss: 0.0181, Accuracy: 100.00%
Epoch [7/10], Loss: 0.0044, Accuracy: 100.00%
Epoch [8/10], Loss: 0.0088, Accuracy: 100.00%
Epoch [9/10], Loss: 0.0035, Accuracy: 100.00%
Epoch [10/10], Loss: 0.0027, Accuracy: 100.00%


In [2]:
# Print out the parameters and their shapes
for name, param in model.named_parameters():
    if param.requires_grad:
        print(f"Parameter name: {name}")
        print(f"Shape: {param.shape}")
        print(f"Number of parameters: {param.numel()}")
        print("-" * 50)

# Total number of parameters
total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Total number of trainable parameters: {total_params}")


Parameter name: conv1.weight
Shape: torch.Size([32, 1, 3, 3])
Number of parameters: 288
--------------------------------------------------
Parameter name: conv1.bias
Shape: torch.Size([32])
Number of parameters: 32
--------------------------------------------------
Parameter name: conv2.weight
Shape: torch.Size([64, 32, 3, 3])
Number of parameters: 18432
--------------------------------------------------
Parameter name: conv2.bias
Shape: torch.Size([64])
Number of parameters: 64
--------------------------------------------------
Parameter name: fc1.weight
Shape: torch.Size([128, 256])
Number of parameters: 32768
--------------------------------------------------
Parameter name: fc1.bias
Shape: torch.Size([128])
Number of parameters: 128
--------------------------------------------------
Parameter name: fc2.weight
Shape: torch.Size([1, 128])
Number of parameters: 128
--------------------------------------------------
Parameter name: fc2.bias
Shape: torch.Size([1])
Number of parameters: 

In [15]:
# 테스트 평가
model.eval()
all_labels = []
all_preds = []
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.unsqueeze(1)
        outputs = model(images)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(outputs.cpu().numpy())

# 평가 지표 계산
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score, average_precision_score

all_labels = np.array(all_labels)
all_preds = np.array(all_preds)

# 이진 분류 결과 계산
test_accuracy = accuracy_score(all_labels, all_preds > 0.5)
test_roc_auc = roc_auc_score(all_labels, all_preds)
test_precision = precision_score(all_labels, all_preds > 0.5)
test_recall = recall_score(all_labels, all_preds > 0.5)
test_f1 = f1_score(all_labels, all_preds > 0.5)
test_auprc = average_precision_score(all_labels, all_preds)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")
print(f"AUROC: {test_roc_auc:.4f}")
print(f"Precision: {test_precision:.4f}")
print(f"Recall: {test_recall:.4f}")
print(f"F1 Score: {test_f1:.4f}")
print(f"AUPRC: {test_auprc:.4f}")


Test Accuracy: 100.00%
AUROC: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
AUPRC: 1.0000
